In [1]:
import os
import numpy as np 
import pandas as pd 
import seaborn as sns 
from PIL import Image 
from datetime import datetime 
import matplotlib.pyplot as plt 
from wordcloud import WordCloud, STOPWORDS
from pathlib import Path
from math import sqrt
from tqdm import tqdm
tqdm.pandas()
import glob

# Read Data

In [2]:
data_path = Path("/Users/me./Desktop/Spring '22 /Machine Learning/Project/h-and-m-personalized-fashion-recommendations/")

In [3]:
art = pd.read_csv("/Users/me./Desktop/Spring '22 /Machine Learning/Project/h-and-m-personalized-fashion-recommendations/articles.csv")
cust = pd.read_csv("/Users/me./Desktop/Spring '22 /Machine Learning/Project/h-and-m-personalized-fashion-recommendations/customers.csv")
sampl = pd.read_csv("/Users/me./Desktop/Spring '22 /Machine Learning/Project/h-and-m-personalized-fashion-recommendations/sample_submission.csv")
trans = pd.read_csv("/Users/me./Desktop/Spring '22 /Machine Learning/Project/h-and-m-personalized-fashion-recommendations/transactions_train.csv")

# Innerjoin Data

In [4]:
alldf = pd.merge(trans, art, on= 'article_id', how= 'inner')

sample data 5% of population size 

In [5]:
sdf = alldf.sample(frac =.05) 

In [6]:
sdf.head()

,t_dat,customer_id,article_id,price,sales_channel_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,...,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc
25527092,2020-08-12,7fb5842440818e3590cf7ee80d4148d6b8b7d80f768a63...,811907004,0.018627,2,811907,Sporty spice top,298,Bikini top,Swimwear,...,Swimwear,B,Lingeries/Tights,1,Ladieswear,60,"Womens Swimwear, beachwear",1018,Swimwear,Fully lined bikini top with removable inserts ...
18411104,2019-07-01,c9e5c64e49cfb882f6760e06bde653682fb3b12c61b29d...,774543002,0.037271,2,774543,Premium BLAKE jersey tee,252,Sweater,Garment Upper body,...,Jersey/Knitwear Premium,A,Ladieswear,1,Ladieswear,14,Womens Premium,1003,Knitwear,Fitted T-shirt in ribbed jersey made from a si...
10419787,2019-08-14,fe71f1397b090de9227bcbb9dc304ae75c00fc8ba22301...,721589001,0.030051,2,721589,Tortoise dress,265,Dress,Garment Full body,...,Jersey,A,Ladieswear,1,Ladieswear,11,Womens Tailoring,1005,Jersey Fancy,"One-shoulder, knee-length dress in softly drap..."
1568910,2019-12-27,d958aa81ac61c1986eb0baafaea5eb1ea318af7ce4b96e...,399087015,0.018627,1,399087,S.Skinny L.W Epic,272,Trousers,Garment Lower body,...,Denim Trousers,D,Divided,2,Divided,57,Ladies Denim,1016,Trousers Denim,"5-pocket, low-rise jeans in washed, superstret..."
20795922,2020-09-20,2edfb6c738efc0707d24e76f86888c81d903e809b7746a...,762846006,0.025407,2,762846,Lucy blouse,259,Shirt,Garment Upper body,...,Blouse,A,Ladieswear,1,Ladieswear,11,Womens Tailoring,1010,Blouses,Long-sleeved blouse in woven fabric with a col...


# Preprocessing

reset index of new dataframe; drop null values; get rid of index and level_0 columns

In [7]:
print(len(sdf))
sdf = sdf.reset_index()
sdf = sdf.dropna()
sdf = sdf.drop(columns = 'index')


1589416


show new merged dataset

In [8]:
sdf.head()

,t_dat,customer_id,article_id,price,sales_channel_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,...,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc
0,2020-08-12,7fb5842440818e3590cf7ee80d4148d6b8b7d80f768a63...,811907004,0.018627,2,811907,Sporty spice top,298,Bikini top,Swimwear,...,Swimwear,B,Lingeries/Tights,1,Ladieswear,60,"Womens Swimwear, beachwear",1018,Swimwear,Fully lined bikini top with removable inserts ...
1,2019-07-01,c9e5c64e49cfb882f6760e06bde653682fb3b12c61b29d...,774543002,0.037271,2,774543,Premium BLAKE jersey tee,252,Sweater,Garment Upper body,...,Jersey/Knitwear Premium,A,Ladieswear,1,Ladieswear,14,Womens Premium,1003,Knitwear,Fitted T-shirt in ribbed jersey made from a si...
2,2019-08-14,fe71f1397b090de9227bcbb9dc304ae75c00fc8ba22301...,721589001,0.030051,2,721589,Tortoise dress,265,Dress,Garment Full body,...,Jersey,A,Ladieswear,1,Ladieswear,11,Womens Tailoring,1005,Jersey Fancy,"One-shoulder, knee-length dress in softly drap..."
3,2019-12-27,d958aa81ac61c1986eb0baafaea5eb1ea318af7ce4b96e...,399087015,0.018627,1,399087,S.Skinny L.W Epic,272,Trousers,Garment Lower body,...,Denim Trousers,D,Divided,2,Divided,57,Ladies Denim,1016,Trousers Denim,"5-pocket, low-rise jeans in washed, superstret..."
4,2020-09-20,2edfb6c738efc0707d24e76f86888c81d903e809b7746a...,762846006,0.025407,2,762846,Lucy blouse,259,Shirt,Garment Upper body,...,Blouse,A,Ladieswear,1,Ladieswear,11,Womens Tailoring,1010,Blouses,Long-sleeved blouse in woven fabric with a col...


create a season column

In [9]:
#get month of purchase and create new column named month 
sdf['month'] = sdf['t_dat'].apply(lambda x: (datetime.strptime(x, '%Y-%m-%d')).date().month)

In [10]:
#make all values that meet criteria 1 else 1 (1: summer, 0: not summer)
def summer(data):
    if data['month'] == 6:
        val = 1
    elif data['month'] == 7:
        val = 1
    elif data['month'] == 8:
        val = 1
    else:
        val = 0
    return val

In [11]:
#apply funcion and create summer column 
sdf['summer'] = sdf.apply(summer, axis =1)

view new df

In [13]:
sdf.head(2)

,t_dat,customer_id,article_id,price,sales_channel_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,...,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc,month,summer
0,2020-08-12,7fb5842440818e3590cf7ee80d4148d6b8b7d80f768a63...,811907004,0.018627,2,811907,Sporty spice top,298,Bikini top,Swimwear,...,Lingeries/Tights,1,Ladieswear,60,"Womens Swimwear, beachwear",1018,Swimwear,Fully lined bikini top with removable inserts ...,8,1
1,2019-07-01,c9e5c64e49cfb882f6760e06bde653682fb3b12c61b29d...,774543002,0.037271,2,774543,Premium BLAKE jersey tee,252,Sweater,Garment Upper body,...,Ladieswear,1,Ladieswear,14,Womens Premium,1003,Knitwear,Fitted T-shirt in ribbed jersey made from a si...,7,1


In [23]:
sdf.columns

Index(['t_dat', 'customer_id', 'article_id', 'price', 'sales_channel_id',
       'product_code', 'prod_name', 'product_type_no', 'product_type_name',
       'product_group_name', 'graphical_appearance_no',
       'graphical_appearance_name', 'colour_group_code', 'colour_group_name',
       'perceived_colour_value_id', 'perceived_colour_value_name',
       'perceived_colour_master_id', 'perceived_colour_master_name',
       'department_no', 'department_name', 'index_code', 'index_name',
       'index_group_no', 'index_group_name', 'section_no', 'section_name',
       'garment_group_no', 'garment_group_name', 'detail_desc', 'month',
       'summer'],
      dtype='object')

drop columns create x and y variables 

In [47]:
len(list(sdf['index_name'].unique()))

10

In [43]:
x = sdf.drop(columns = ['t_dat', 'customer_id','article_id','price','sales_channel_id','product_code', 'prod_name','product_type_no','product_type_name', 'graphical_appearance_no', 'graphical_appearance_name', 'colour_group_code','perceived_colour_value_id', 'perceived_colour_master_id','perceived_colour_master_name','department_no','department_name','index_code','index_group_no','index_group_name','section_no','section_name', 'garment_group_no', 'garment_group_name', 'detail_desc', 'month', 'summer'])
y = sdf['summer']

In [56]:
x.head()

,product_group_name,colour_group_name,perceived_colour_value_name,index_name
0,Swimwear,Other Yellow,Bright,Lingeries/Tights
1,Garment Upper body,Off White,Dusty Light,Ladieswear
2,Garment Full body,Black,Dark,Ladieswear
3,Garment Lower body,Dark Blue,Medium Dusty,Divided
4,Garment Upper body,Black,Dark,Ladieswear


In [45]:
X = pd.get_dummies(x)

# Standardize

In [51]:
from sklearn.preprocessing import StandardScaler
colnames = list(X.columns)
std = StandardScaler().fit_transform(X)
xstd = pd.DataFrame(std, columns = colnames)

In [52]:
xstd.head()

,product_group_name_Accessories,product_group_name_Bags,product_group_name_Cosmetic,product_group_name_Furniture,product_group_name_Garment Full body,product_group_name_Garment Lower body,product_group_name_Garment Upper body,product_group_name_Garment and Shoe care,product_group_name_Interior textile,product_group_name_Items,...,index_name_Baby Sizes 50-98,"index_name_Children Accessories, Swimwear",index_name_Children Sizes 134-170,index_name_Children Sizes 92-140,index_name_Divided,index_name_Ladies Accessories,index_name_Ladieswear,index_name_Lingeries/Tights,index_name_Menswear,index_name_Sport
0,-0.231016,-0.015472,-0.006407,-0.003276,-0.355026,-0.533541,-0.809314,-0.002973,-0.000795,-0.013346,...,-0.089806,-0.058624,-0.127067,-0.105991,-0.538899,-0.243549,-0.835551,2.175868,-0.243357,-0.20174
1,-0.231016,-0.015472,-0.006407,-0.003276,-0.355026,-0.533541,1.235614,-0.002973,-0.000795,-0.013346,...,-0.089806,-0.058624,-0.127067,-0.105991,-0.538899,-0.243549,1.196815,-0.459587,-0.243357,-0.20174
2,-0.231016,-0.015472,-0.006407,-0.003276,2.816694,-0.533541,-0.809314,-0.002973,-0.000795,-0.013346,...,-0.089806,-0.058624,-0.127067,-0.105991,-0.538899,-0.243549,1.196815,-0.459587,-0.243357,-0.20174
3,-0.231016,-0.015472,-0.006407,-0.003276,-0.355026,1.874270,-0.809314,-0.002973,-0.000795,-0.013346,...,-0.089806,-0.058624,-0.127067,-0.105991,1.855635,-0.243549,-0.835551,-0.459587,-0.243357,-0.20174
4,-0.231016,-0.015472,-0.006407,-0.003276,-0.355026,-0.533541,1.235614,-0.002973,-0.000795,-0.013346,...,-0.089806,-0.058624,-0.127067,-0.105991,-0.538899,-0.243549,1.196815,-0.459587,-0.243357,-0.20174


# PCA

In [53]:
from sklearn.decomposition import PCA 
pca = PCA()
pca.fit(xstd)
pca.explained_variance_ratio_.cumsum()

array([0.03335722, 0.06300778, 0.08978239, 0.11423625, 0.13752868,
       0.16079214, 0.18281563, 0.20316674, 0.22184876, 0.23901563,
       0.25527969, 0.27020819, 0.28472803, 0.29876037, 0.31263457,
       0.32581271, 0.3386257 , 0.35131265, 0.36393885, 0.37627624,
       0.38847503, 0.40059813, 0.41268532, 0.4246509 , 0.43658072,
       0.44845359, 0.46031527, 0.47216117, 0.48397517, 0.49575507,
       0.50752715, 0.51927898, 0.53102005, 0.54274922, 0.55446429,
       0.56617234, 0.57786844, 0.58955851, 0.60124169, 0.61291933,
       0.62458736, 0.6362515 , 0.64790709, 0.65956069, 0.67120755,
       0.68285142, 0.69449025, 0.70612646, 0.71775891, 0.72938875,
       0.74101773, 0.75264526, 0.76426717, 0.77588044, 0.78748563,
       0.79906114, 0.8106117 , 0.82213206, 0.83364049, 0.84512999,
       0.85656757, 0.8679542 , 0.87918206, 0.89034138, 0.90130356,
       0.91215889, 0.92286568, 0.93313307, 0.943268  , 0.95305372,
       0.96266353, 0.97167596, 0.97988207, 0.98676453, 0.99183

In [54]:
pca = PCA(n_components= 70) #explains over 95% of the variance in the data 
pcax = pca.fit_transform(xstd)

In [55]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

xtrain, xtest, ytrain, ytest = train_test_split(pcax, y, test_size = .4, random_state= 5)
rf = RandomForestClassifier()
rf.fit(xtrain, ytrain)

ypred = rf.predict(xtest)
print(round(accuracy_score(ytest, ypred),2))

0.72
